In [13]:
import numpy as np

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Fully Connected Neural Network with 3 hidden layers 

In [19]:
# Load SVHNDigit data

from SVHNDigit.generic import read_dataset

data_dir = '../data'
train_filename = 'train_32x32.mat'
test_filename = 'test_32x32.mat'

train_X, train_y, val_X, val_y, test_X, test_y = \
    read_dataset(data_dir, train_filename, test_filename, val_size=20132, reshape=True)

Training data shape:  (53125, 3072) (53125, 10)
Validation data shape:  (20132, 3072) (20132, 10)
Test data shape:  (26032, 3072) (26032, 10)


In [21]:
# Train a Fully Connected NN with 3 hidden layers of sizes 64, 128 and 256

from SVHNDigit.models.fcnn_relu_64_128_256_10_softmax.model import FCNN
from SVHNDigit.generic import train_model

input_dim = train_X.shape[1]

lr = 1.865e-3
reg_factor = 2e-6
dropout_param = 0.0
momentum = 0.0

model_define_params = {'reg_factor': reg_factor, 
                       'init': 'glorot_normal',
                       'use_dropout': False,
                       'dropout_param': dropout_param, 
                       'use_batchnorm': True}    

fcnn_model = FCNN(model_define_params, input_dim)
fcnn_model = fcnn_model.define(verbose=0)

model_train_params = {'loss': 'categorical_crossentropy', 
                      'optimizer': 'adam', 
                      'lr': lr,
                      'momentum': momentum,
                      'nesterov': False,
                      'metrics': ['accuracy'], 
                      'batch_size': 128, 
                      'nb_epochs': 20}

train_model(fcnn_model, model_train_params, train_X, train_y, 
            val_X, val_y, verbose=1, tb_logs=True)

Epoch 1/20
53125/53125 [==============================] - 25s - loss: 1.3077 - acc: 0.5717    
Epoch 2/20
53125/53125 [==============================] - 23s - loss: 0.9213 - acc: 0.7090    
Epoch 3/20
53125/53125 [==============================] - 23s - loss: 0.7958 - acc: 0.7486    
Epoch 4/20
53125/53125 [==============================] - 24s - loss: 0.7266 - acc: 0.7713    
Epoch 5/20
53125/53125 [==============================] - 23s - loss: 0.7255 - acc: 0.7726    
Epoch 6/20
53125/53125 [==============================] - 23s - loss: 0.6541 - acc: 0.7961    
Epoch 7/20
53125/53125 [==============================] - 26s - loss: 0.6516 - acc: 0.7948    
Epoch 8/20
53125/53125 [==============================] - 27s - loss: 0.6061 - acc: 0.8109    
Epoch 9/20
53125/53125 [==============================] - 23s - loss: 0.5605 - acc: 0.8261    
Epoch 10/20
53125/53125 [==============================] - 24s - loss: 0.5359 - acc: 0.8335    
Epoch 11/20
53125/53125 [========================

In [14]:
from keras.utils import np_utils


pred_y = fcnn_model.predict_classes(test_X, batch_size=128, verbose=1)
print "Test Accuracy: %0.2f" % (np_utils.accuracy(pred_y, np.argmax(test_y, axis=1)))

26032/26032 [==============================] - 4s     
Test Accuracy: 0.71


### Convolutional Neural Network - [conv-relu-pool] x 2 - [conv-relu] x 1 - [affine-relu] x 2 - [affine-softmax]

In [15]:
# Load SVHNDigit data

from SVHNDigit.generic import read_dataset

data_dir = '../data'
train_filename = 'train_32x32.mat'
test_filename = 'test_32x32.mat'

train_X, train_y, val_X, val_y, test_X, test_y = \
    read_dataset(data_dir, train_filename, test_filename, val_size=20132, reshape=False)

Training data shape:  (53125, 3, 32, 32) (53125, 10)
Validation data shape:  (20132, 3, 32, 32) (20132, 10)
Test data shape:  (26032, 3, 32, 32) (26032, 10)


In [18]:
from SVHNDigit.models.fcnn_relu_64_128_256_10_softmax.model import FCNN
from SVHNDigit.models.cnn.model import CNN_1
from SVHNDigit.generic import train_model

input_dim = train_X.shape[1:]

#num_samples = 9600
#train_X_small = train_X[0:num_samples, :, :, :]
#train_y_small = train_y[0:num_samples]

lr = 1e-1
reg_factor = 2e-6
dropout_param = 0.05
momentum = 0.2

model_define_params = {'reg_factor': reg_factor, 
                       'init': 'glorot_normal',
                       'use_dropout': False,
                       'dropout_param': dropout_param, 
                       'use_batchnorm': True}    

cnn_model = CNN_1(model_define_params, input_dim)
cnn_model = cnn_model.define(verbose=0)

model_train_params = {'loss': 'categorical_crossentropy', 
                      'optimizer': 'adam', 
                      'lr': lr,
                      'momentum': momentum,
                      'nesterov': False,
                      'metrics': ['accuracy'], 
                      'batch_size': 128, 
                      'nb_epochs': 20}

train_model(cnn_model, model_train_params, train_X, train_y, 
            val_X, val_y, verbose=1)

Epoch 1/20
 8576/53125 [===>..........................] - ETA: 3012s - loss: 7.5644 - acc: 0.1705

KeyboardInterrupt: 